# EBAC - Regressão II - regressão múltipla

## Tarefa II

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.tree import plot_tree

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df = df.drop(['data_ref','id_cliente','Unnamed: 0'],axis=1)
df.sample(2)

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
14916,F,False,True,0,Pensionista,Secundário,União,Casa,57,NaN,2.0,913.15
6153,F,False,True,1,Assalariado,Secundário,Casado,Casa,45,1.720548,3.0,2085.68


In [4]:
df = df.dropna()
df.isna().sum()

sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
dtype: int64

In [5]:
df['posse_de_veiculo'] = df['posse_de_veiculo'].astype(int)
df['posse_de_imovel'] = df['posse_de_imovel'].astype(int)

In [6]:
df = pd.get_dummies(df, columns=['tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia','sexo'], drop_first=True)

In [7]:
df.columns

Index(['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'renda',
       'tipo_renda_Bolsista', 'tipo_renda_Empresário',
       'tipo_renda_Pensionista', 'tipo_renda_Servidor público',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior completo', 'educacao_Superior incompleto',
       'estado_civil_Separado', 'estado_civil_Solteiro', 'estado_civil_União',
       'estado_civil_Viúvo', 'tipo_residencia_Casa',
       'tipo_residencia_Com os pais', 'tipo_residencia_Comunitário',
       'tipo_residencia_Estúdio', 'tipo_residencia_Governamental', 'sexo_M'],
      dtype='object')

In [8]:
df = df.rename(columns={
    'tipo_renda_Servidor público': 'tipo_renda_Servidor_público',
    'educacao_Pós graduação': 'educacao_Pós_graduação',
    'educacao_Superior completo': 'educacao_Superior_completo',
    'educacao_Superior incompleto': 'educacao_Superior_incompleto',
    'tipo_residencia_Com os pais': 'tipo_residencia_Com_pais'
})

---

In [9]:
# Separe a base em treinamento e teste (25% para teste, 75% para treinamento)
df_train, df_test = train_test_split(df, test_size = 0.25, random_state = 42)

In [10]:
modelo = '''renda ~ tipo_renda_Empresário
                    + tipo_renda_Pensionista
                    + tipo_renda_Servidor_público
                    + educacao_Pós_graduação
                    + educacao_Secundário
                    + educacao_Superior_completo
                    + educacao_Superior_incompleto
                    + estado_civil_Separado
                    + estado_civil_Solteiro
                    + estado_civil_União
                    + estado_civil_Viúvo
                    + idade 
                    + qt_pessoas_residencia 
                    + qtd_filhos 
                    + posse_de_veiculo 
                    + posse_de_imovel 
                    + tipo_residencia_Casa
                    + tipo_residencia_Com_pais
                    + tipo_residencia_Comunitário
                    + tipo_residencia_Estúdio
                    + tipo_residencia_Governamental
                    + tempo_emprego
                    + sexo_M
                    '''
y_train_OLS, x_train_OLS = patsy.dmatrices(formula_like = modelo, data = df_train)
y_test_OLS, x_test_OLS = patsy.dmatrices(formula_like = modelo, data = df_test)

In [11]:
print(f' x treino: {x_train_OLS.shape}')
print(f' x teste:  {x_test_OLS.shape}')
print(f' y treino: {y_train_OLS.shape}')
print(f' y teste:  {y_test_OLS.shape}')

 x treino: (9320, 24)
 x teste:  (3107, 24)
 y treino: (9320, 1)
 y teste:  (3107, 1)


In [12]:
x_train_OLS = sm.add_constant(x_train_OLS)
x_test_OLS = sm.add_constant(x_test_OLS)

In [13]:
# Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o na base de testes. Qual o melhor modelo?
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    modelo_OLS = sm.OLS(y_train_OLS, x_train_OLS)
    reg = modelo_OLS.fit_regularized(method = 'elastic_net', 
                                     refit = True,
                                     L1_wt = 0,  # ridge fit
                                     alpha = alpha)

    
    y_pred = reg.predict(x_test_OLS)  # predição
    r2 = r2_score(y_test_OLS, y_pred)

    r2_y_pred.append(r2)

df_r2_ridge = pd.DataFrame({'alpha': alphas, 'R2': r2_y_pred}).sort_values(by='R2', ascending = False)
df_r2_ridge


,alpha,R2
3,0.010,0.299272
2,0.005,0.298945
1,0.001,0.298343
0,0.000,0.297970
4,0.050,0.296780
5,0.100,0.289524


---

In [14]:
# Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    modelo_OLS = sm.OLS(y_train_OLS, x_train_OLS)
    reg = modelo_OLS.fit_regularized(method = 'elastic_net', 
                                     refit = True,
                                     L1_wt = 1, # lasso
                                     alpha = alpha)

    
    y_pred = reg.predict(x_test_OLS)  # predição
    r2 = r2_score(y_test_OLS, y_pred)

    r2_y_pred.append(r2)

df_r2_lasso = pd.DataFrame({'alpha': alphas, 'R2': r2_y_pred}).sort_values(by='R2', ascending = False)
df_r2_lasso


#3 ok

,alpha,R2
5,0.100,0.298512
3,0.010,0.298121
0,0.000,0.297970
1,0.001,0.297970
2,0.005,0.297970
4,0.050,0.297970


In [15]:
# Rode um modelo stepwise. Avalie o na vase de testes. Qual o melhor resultado?

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed = False
  
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
        
    
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()

        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included


y_train_OLS, x_train_OLS = patsy.dmatrices(formula_like=modelo, data=df_train)
y_test_OLS, x_test_OLS = patsy.dmatrices(formula_like=modelo, data=df_test)

#convertendo para Df para eu acessar os valores
x_train_OLS = pd.DataFrame(x_train_OLS, columns=x_train_OLS.design_info.column_names)
x_test_OLS = pd.DataFrame(x_test_OLS, columns=x_test_OLS.design_info.column_names)
y_train_OLS = pd.Series(y_train_OLS.ravel(), name='renda')
y_test_OLS = pd.Series(y_test_OLS.ravel(), name='renda')


result = stepwise_selection(x_train_OLS, y_train_OLS)

model = sm.OLS(y_train_OLS, sm.add_constant(x_train_OLS[result])).fit()

y_pred = model.predict(sm.add_constant(x_test_OLS[result]))
r2 = r2_score(y_test_OLS, y_pred)
print(f'R²: {r2}')

model.summary()

Add  Intercept                      with p-value 0.0
Add  tempo_emprego                  with p-value 0.0
Add  sexo_M[T.True]                 with p-value 1.26513e-242
Add  tipo_renda_Empresário[T.True]  with p-value 4.12304e-06
Add  idade                          with p-value 2.37794e-05
Add  educacao_Superior_completo[T.True] with p-value 0.00080423
R²: 0.2968993711320578


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     605.3
Date:                Mon, 08 Jul 2024   Prob (F-statistic):               0.00
Time:                        17:48:20   Log-Likelihood:                -97077.
No. Observations:                9320   AIC:                         1.942e+05
Df Residuals:                    9314   BIC:                         1.942e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                          -2719.6957    410.321     -6.628      0.000   -3524.014   -1915.377
tempo_emprego                        570.3956     13.211     43.176      0.000     544.499     596.292
sexo_M[T.True]                      6191.2357    177.207     34.938      0.000    5843.872    6538.599
tipo_renda_Empresário[T.True]        840.5513    188.701      4.454      0.000     470.657    1210.446
idade                                 42.8900      9.515      4.508      0.000      24.239      61.541
educacao_Superior_completo[T.True]   582.1203    173.641      3.352      0.001     241.746     922.495
==============================================================================
Omnibus:                    13569.081   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8080663.709
Skew:                           8.590   Prob(JB):                         0.00
Kurtosis:                     146.225   Cond. No.                         213.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
#### Comparação dos Resultados

##### Regressão LASSO

| alpha | R²       |
|-------|----------|
| 0.100 | 0.298512 |
| 0.010 | 0.298121 |
| 0.000 | 0.297970 |
| 0.001 | 0.297970 |
| 0.005 | 0.297970 |
| 0.050 | 0.297970 |

##### Regularização Ridge

| alpha | R²       |
|-------|----------|
| 0.010 | 0.299272 |
| 0.005 | 0.298945 |
| 0.001 | 0.298343 |
| 0.000 | 0.297970 |
| 0.050 | 0.296780 |
| 0.100 | 0.289524 |

##### Modelo Stepwise

|     | R²         |
|-----|------------|
|     | 0.296899   |

#### Conclusão

O modelo de regularização Ridge com alpha = 0.01 obteve o melhor desempenho, com um R² de 0.299272. 

Portanto, entre os três métodos, a regularização Ridge fornece o melhor modelo, baseado no valor de R².

In [28]:
#nova matriz
y_train, x_train = patsy.dmatrices(formula_like=modelo, data=df_train)
y_test, x_test = patsy.dmatrices(formula_like=modelo, data=df_test)

x_train = pd.DataFrame(x_train, columns=x_train.design_info.column_names)
x_test = pd.DataFrame(x_test, columns=x_test.design_info.column_names)
y_train = y_train.ravel()
y_test = y_test.ravel()


scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

tree_regressor = DecisionTreeRegressor(random_state=7)
tree_regressor.fit(x_train_scaled, y_train)

y_pred = tree_regressor.predict(x_test_scaled)

r2 = r2_score(y_test, y_pred)
print(f'R² da Árvore de Regressão: {r2}')

print('0'*10)
tree_regressor


R² da Árvore de Regressão: 0.24139506679683742
0000000000


DecisionTreeRegressor(random_state=7)

In [ ]:
# tree_regressor = DecisionTreeRegressor(random_state=7)
# tree_regressor.fit(x_train_scaled, y_train)

# plt.figure(figsize=(20, 10))
# plot_tree(tree_regressor, feature_names=x_train.columns, filled=True, rounded=True)
# plt.title("Árvore de Regressão")
# plt.show()


### Conclusão da Árvore de Regressão

No contexto da atividade, o melhor resultado encontrado foi testando o `random_state` e atribuindo o valor inicial de '7', chegando em um R² de 0.2413. O problema foi plotar a figura, pois ficou imensa.